In [ ]:
import gym
import numpy as np
import pygame
import random

In [ ]:
# Définir l'environnement des bonhommes
class BonhommeEnv(gym.Env):
    def __init__(self):
        super(BonhommeEnv, self).__init__()

        # Définition des actions possibles (ex: se déplacer dans une grille)
        self.action_space = gym.spaces.Discrete(4)  # 4 actions possibles : Haut, Bas, Gauche, Droite

        # Définition de l'état (par exemple, position du bonhomme dans une grille 5x5)
        self.observation_space = gym.spaces.Box(low=0, high=4, shape=(2,), dtype=np.int32)

        # Initialiser la position du bonhomme
        self.state = np.array([0, 0])  # Position initiale

        # Récompenses
        self.reward = 0

        # Taille de la grille (pour pygame)
        self.grid_size = 5
        self.cell_size = 100  # Taille d'une cellule dans la grille

        # Initialiser Pygame
        pygame.init()
        self.screen = pygame.display.set_mode((self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        pygame.display.set_caption('Simulation des Bonhommes')
        self.clock = pygame.time.Clock()

    def step(self, action):
        # Appliquer l'action et mettre à jour la position
        if action == 0:  # Haut
            self.state[1] = max(self.state[1] - 1, 0)
        elif action == 1:  # Bas
            self.state[1] = min(self.state[1] + 1, 4)
        elif action == 2:  # Gauche
            self.state[0] = max(self.state[0] - 1, 0)
        elif action == 3:  # Droite
            self.state[0] = min(self.state[0] + 1, 4)

        # Calculer la récompense 
        done = bool(self.state[0] == 4 and self.state[1] == 4)
        self.reward = 1 if done else -0.1

        return self.state, self.reward, done, {}

    def reset(self):
        # Réinitialiser l'environnement
        self.state = np.array([0, 0])
        return self.state

    def render(self):
        # Couleurs
        white = (255, 255, 255)
        black = (0, 0, 0)
        blue = (0, 0, 255)

        # Remplir l'écran avec du blanc
        self.screen.fill(white)

        # Dessiner la grille
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(self.screen, black, rect, 1)  # Dessiner le contour de la case

        # Dessiner le bonhomme (cercle)
        bonhomme_x = self.state[0] * self.cell_size + self.cell_size // 2
        bonhomme_y = self.state[1] * self.cell_size + self.cell_size // 2
        pygame.draw.circle(self.screen, blue, (bonhomme_x, bonhomme_y), self.cell_size // 4)

        # Mettre à jour l'affichage
        pygame.display.flip()

    def close(self):
        pygame.quit()

# Utiliser l'environnement
env = BonhommeEnv()

# Exemple de boucle d'interaction
for episode in range(10):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Gestion des événements Pygame
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                env.close()

        action = env.action_space.sample()  # Choisir une action aléatoire
        state, reward, done, info = env.step(action)
        total_reward += reward
        env.render()

        env.clock.tick(2)  # Limiter la vitesse de la simulation à 2 images par seconde

    print(f"Épisode {episode + 1} terminé avec un total de récompenses de {total_reward}")

env.close()  # Fermer l'environnement après utilisation